# Goal of Notebook

Create a transformer up to the code of 'Attention is all you need' for a case of giving it an input sequence and it would have to try to predict the output.<br>
Processes are per - timestep ( note : not per - token )

Done Task:
Positional Encoding

Current Task:
Embeddings of input(does it work for words and numbers?)

Next Task:
Attention (It is copied input, x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask)))
Multi head Attention
Layer Norm

In [2]:
from torch import nn
import torch
import numpy as np
import math
import torch.nn.functional as f
import copy


/home/nkolln/mt/vis10/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
#Input is a single list of tensors
#output is an array of size list by dimension
#This is constant?? Simpy add to layer
def positional_encoding(dim,max_len=1000):
    arr_pe = torch.zeros(max_len,dim)
    #Creates an array for the positions of rows
    arr_pos = torch.arange(0,max_len).unsqueeze(1)
    #Sine and cosine are applied an alternating dims so half are generated(Sin and cos apply to same subset basically to have the different outputs)
    div_term = torch.exp(torch.arange(0, dim, 2) * -(math.log(10000.0) / dim))
    #basically multipleis the 0-max len values by the div term which is unique per dim
    temp = arr_pos*div_term
    arr_pe[:,0::2]= torch.sin(temp)
    arr_pe[:,1::2]= torch.cos(temp)
    return(arr_pe.unsqueeze(0))

def scaled_dot_product_attention(q,k,v,dim):
    q_k = torch.matmul(q,k.T)
    q_k_scale = q_k * math.sqrt(dim)
    #can add a mask here

    q_k_scale = f.softmax(q_k_scale,dim=1)
    attn = torch.matmul(q_k_scale,v)
    return(attn)

def clones(component,N):
    return(nn.ModuleList([copy.deepcopy(component)for _ in range(0,N)]))


pe = positional_encoding(20)
    

In [51]:
#Embedding
#Embedding only works for discrete (Can we just set a ridiculously large embedding and treat continuous values as discrete)(Can we use convolutional kernels possibly?)
#Embedding embedds and then creates linear layers to make the v,k,q from same data,then process through scaled product attention, concat and apply final linear.
class EmbeddingLayer(nn.Module):
    def __init__(self,dic_size,dim,size):
        super().__init__()
        # self.dc = dic_size
        self.dim = dim
        self.embedding = nn.Embedding(dic_size,dim)
        self.pe = positional_encoding(dim,size)

    def forward(self,x):
        x = self.embedding(x) 
        #x = x* math.sqrt(self.dim)
        print(x.shape)
        #x = x + self.pe
        return x,pe



class AttentionHead(nn.Module):
    def __init__(self,l_layer):
        super().__init__()
        
        self.l1 = nn.Linear(l_layer,l_layer)
        self.l2 = nn.Linear(l_layer,l_layer)
        self.l3 = nn.Linear(l_layer,l_layer)
    def forward(self,x):
        q = f.relu(self.l1(x))
        k = f.relu(self.l2(x))
        v = f.relu(self.l3(x))
        return(x,q,k,v)

class MultiHeadAttention(nn.Module):
    def __init__(self):
        super().__init__()


input = torch.LongTensor([[1,2,4],[4,3,2]])
data = input
dic_size = 1000;embed_size=512;attn_layer_size = embed_size
el = EmbeddingLayer(dic_size,embed_size,input.shape[1])
# print(type(data))
embed,pe = el(data)
print(embed)
attnhead = AttentionHead(attn_layer_size)
# print(attnhead(embed)[1])
# attnhead(embed)[1]

torch.Size([2, 3, 512])
tensor([[[-1.0572, -1.3308,  0.5802,  ..., -0.6959,  1.3029, -0.0801],
         [ 0.4477, -1.1618, -0.9530,  ...,  1.3338,  0.7754,  1.1468],
         [-1.7042,  0.7498,  0.5346,  ...,  0.1864, -0.6773, -1.8943]],

        [[-1.7042,  0.7498,  0.5346,  ...,  0.1864, -0.6773, -1.8943],
         [ 0.2036,  1.9969, -0.2026,  ..., -1.8149,  0.2619,  0.6535],
         [ 0.4477, -1.1618, -0.9530,  ...,  1.3338,  0.7754,  1.1468]]],
       grad_fn=<EmbeddingBackward0>)


In [5]:
def attention(query, key, value, mask=None, dropout=None):
    "Compute 'Scaled Dot Product Attention'"
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = scores.softmax(dim=-1)
    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, value), p_attn
class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1):
        "Take in model size and number of heads."
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0
        # We assume d_v always equals d_k
        self.d_k = d_model // h
        self.h = h
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, query, key, value, mask=None):
        "Implements Figure 2"
        if mask is not None:
            # Same mask applied to all h heads.
            mask = mask.unsqueeze(1)
        nbatches = query.size(0)

        # 1) Do all the linear projections in batch from d_model => h x d_k
        query, key, value = [
            lin(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
            for lin, x in zip(self.linears, (query, key, value))
        ]
        print(query.shape)

        # 2) Apply attention on all the projected vectors in batch.
        x, self.attn = attention(
            query, key, value, mask=mask, dropout=self.dropout
        )

        # 3) "Concat" using a view and apply a final linear.
        x = (
            x.transpose(1, 2)
            .contiguous()
            .view(nbatches, -1, self.h * self.d_k)
        )
        del query
        del key
        del value
        return self.linears[-1](x)

## Architecture

In [6]:
class EncoderDecoder(nn.Module):
    """
    A standard Encoder-Decoder architecture. Base for this and many
    other models.
    """

    def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
        super(EncoderDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.generator = generator

    def forward(self, src, tgt, src_mask, tgt_mask):
        "Take in and process masked src and target sequences."
        return self.decode(self.encode(src, src_mask), src_mask, tgt, tgt_mask)

    def encode(self, src, src_mask):
        return self.encoder(self.src_embed(src), src_mask)

    def decode(self, memory, src_mask, tgt, tgt_mask):
        return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)

class Encoder(nn.Module):
    "Core encoder is a stack of N layers"

    def __init__(self, layer, N):
        super(Encoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)

    def forward(self, x, mask):
        "Pass the input (and mask) through each layer in turn."
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

class EncoderLayer(nn.Module):
    "Encoder is made up of self-attn and feed forward (defined below)"

    def __init__(self, size, self_attn, feed_forward, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 2)
        self.size = size

    def forward(self, x, mask):
        "Follow Figure 1 (left) for connections."
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
        return self.sublayer[1](x, self.feed_forward)

class Decoder(nn.Module):
    "Generic N layer decoder with masking."

    def __init__(self, layer, N):
        super(Decoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)

    def forward(self, x, memory, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, memory, src_mask, tgt_mask)
        return self.norm(x)

class DecoderLayer(nn.Module):
    "Decoder is made of self-attn, src-attn, and feed forward (defined below)"

    def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
        super(DecoderLayer, self).__init__()
        self.size = size
        self.self_attn = self_attn
        self.src_attn = src_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 3)

    def forward(self, x, memory, src_mask, tgt_mask):
        "Follow Figure 1 (right) for connections."
        m = memory
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, tgt_mask))
        x = self.sublayer[1](x, lambda x: self.src_attn(x, m, m, src_mask))
        return self.sublayer[2](x, self.feed_forward)

In [7]:
class LayerNorm(nn.Module):
    "Construct a layernorm module (See citation for details)."

    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

#Order of components seems not to be important
#This is done to help with the vanishing gradient problem in backpropagation
class SublayerConnection(nn.Module):
    """
    A residual connection followed by a layer norm.
    Note for code simplicity the norm is first as opposed to last.
    """

    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        "Apply residual connection to any sublayer with the same size."
        return x + self.dropout(sublayer(self.norm(x)))

## connections

In [8]:
class Embeddings(nn.Module):
    def __init__(self, d_model, vocab):
        super(Embeddings, self).__init__()
        self.lut = nn.Embedding(vocab, d_model)
        self.d_model = d_model

    def forward(self, x):
        return self.lut(x) * math.sqrt(self.d_model)

class PositionwiseFeedForward(nn.Module):
    "Implements FFN equation."

    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w_2(self.dropout(self.w_1(x).relu()))

class PositionalEncoding(nn.Module):
    "Implement the PE function."

    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(
            torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model)
        )
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer("pe", pe)

    def forward(self, x):
        x = x + self.pe[:, : x.size(1)].requires_grad_(False)
        return self.dropout(x)

class Generator(nn.Module):
    "Define standard linear + softmax generation step."

    def __init__(self, d_model, vocab):
        super(Generator, self).__init__()
        self.proj = nn.Linear(d_model, vocab)

    def forward(self, x):
        return log_softmax(self.proj(x), dim=-1)

In [9]:

def subsequent_mask(size):
    "Mask out subsequent positions."
    attn_shape = (1, size, size)
    subsequent_mask = torch.triu(torch.ones(attn_shape), diagonal=1).type(
        torch.uint8
    )
    return subsequent_mask == 0
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    memory = model.encode(src, src_mask)
    ys = torch.zeros(1, 1).fill_(start_symbol).type_as(src.data)
    for i in range(max_len - 1):
        out = model.decode(
            memory, src_mask, ys, subsequent_mask(ys.size(1)).type_as(src.data)
        )
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.data[0]
        ys = torch.cat(
            [ys, torch.zeros(1, 1).type_as(src.data).fill_(next_word)], dim=1
        )
    return ys

In [ ]:
def make_model(
    src_vocab, tgt_vocab, N=6, d_model=512, d_ff=2048, h=8, dropout=0.1
):
    "Helper: Construct a model from hyperparameters."
    c = copy.deepcopy
    attn = MultiHeadedAttention(h, d_model)
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
    position = PositionalEncoding(d_model, dropout)
    model = EncoderDecoder(
        Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
        Decoder(DecoderLayer(d_model, c(attn), c(attn), c(ff), dropout), N),
        nn.Sequential(Embeddings(d_model, src_vocab), c(position)),
        nn.Sequential(Embeddings(d_model, tgt_vocab), c(position)),
        Generator(d_model, tgt_vocab),
    )

    # This was important from their code.
    # Initialize parameters with Glorot / fan_avg.
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)
    return model

# Testing

In [56]:
class PositionalEncoding(nn.Module):
    "Implement the PE function."

    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(
            torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model)
        )
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer("pe", pe)

    def forward(self, x):
        print(x.shape)
        print(self.pe.shape)
        x = x + self.pe[:, : x.size(1)].requires_grad_(False)
        return self.dropout(x)

class TestClass(nn.Module):
    def __init__(self,position):
        super(TestClass,self).__init__()
        self.layer = nn.Sequential(nn.Embedding(1000, 512),position)
    
    def forward(self,x):
        return self.layer(x)


In [69]:
position = PositionalEncoding(512, 0)
test = TestClass(position)
print(input.shape)
out = test(data)
print(out.shape)
out

torch.Size([2, 3])
torch.Size([2, 3, 512])
torch.Size([1, 5000, 512])
torch.Size([2, 3, 512])


tensor([[[ 0.1705,  2.2181, -0.0212,  ...,  0.8106,  0.7591, -0.2599],
         [-0.0576,  1.6933,  1.4200,  ...,  1.1282, -1.1444,  1.1106],
         [ 2.3810,  0.7858,  0.2128,  ...,  1.8807,  0.8289,  2.0980]],

        [[ 1.4717,  2.2020, -0.7236,  ...,  1.8807,  0.8287,  2.0980],
         [ 3.0392, -0.1269,  1.9517,  ..., -0.2170, -0.0198,  1.2920],
         [ 0.0102,  0.7368,  1.5346,  ...,  1.1282, -1.1442,  1.1106]]],
       grad_fn=<AddBackward0>)

In [53]:
print(pe.shape)
pe[:, : 10].shape

torch.Size([1, 1000, 20])


torch.Size([1, 10, 20])

In [4]:
type(torch.exp(torch.arange(0, 20, 2, dtype=torch.float32) * -(math.log(10000.0) / 20)))

torch.Tensor

In [3]:
torch.arange(0, 20, 2, dtype=torch.float)

tensor([ 0.,  2.,  4.,  6.,  8., 10., 12., 14., 16., 18.])

In [2]:
V=11; batch_size = 30
data = torch.randint(1, V, size=(batch_size, 10))
data[:, 0] = 1
src = data.requires_grad_(False).clone().detach()
tgt = data.requires_grad_(False).clone().detach()

In [10]:
np.where(src!=tgt)

(array([], dtype=int64), array([], dtype=int64))

In [22]:
arr = torch.zeros(4,6)
tmp = (torch.arange(0,17,5).unsqueeze(1)*torch.arange(0,30,10))
print(tmp)
arr[:,0::2]=torch.sin(tmp)
arr[:,1::2]=torch.cos(tmp)
print(arr)

tensor([[  0,   0,   0],
        [  0,  50, 100],
        [  0, 100, 200],
        [  0, 150, 300]])
tensor([[ 0.0000,  1.0000,  0.0000,  1.0000,  0.0000,  1.0000],
        [ 0.0000,  1.0000, -0.2624,  0.9650, -0.5064,  0.8623],
        [ 0.0000,  1.0000, -0.5064,  0.8623, -0.8733,  0.4872],
        [ 0.0000,  1.0000, -0.7149,  0.6993, -0.9998, -0.0221]])


In [78]:
# an Embedding module containing 10 tensors of size 3
embedding = nn.Embedding(100, 3)
# a batch of 2 samples of 4 indices each
input = torch.LongTensor([[1,2,4],[4,3,2]])
input = torch.randn(1)
input = input.long()
embedding(input)
print(embedding(input))



tensor([[0.8151, 1.3661, 1.1932]], grad_fn=<EmbeddingBackward0>)


In [42]:
# an Embedding module containing 10 tensors of size 3
embedding = nn.Embedding(5, 3)
# a batch of 2 samples of 4 indices each
input = torch.LongTensor([[1,2,4,2],[4,3,2,1]])
embedding(input)
print(embedding(input))



tensor([[[ 0.0326,  1.5697,  2.0568],
         [-0.0909, -0.0345, -0.1071],
         [ 0.0105, -0.3593,  0.5579],
         [-0.0909, -0.0345, -0.1071]],

        [[ 0.0105, -0.3593,  0.5579],
         [ 0.5077,  0.0521,  0.0648],
         [-0.0909, -0.0345, -0.1071],
         [ 0.0326,  1.5697,  2.0568]]], grad_fn=<EmbeddingBackward0>)


# Old Code

In [70]:
#Embedding
#Embedding only works for discrete (Can we just set a ridiculously large embedding and treat continuous values as discrete)(Can we use convolutional kernels possibly?)
#Embedding embedds and then creates linear layers to make the v,k,q from same data,then process through scaled product attention, concat and apply final linear.
class EmbeddingLayer(nn.Module):
    def __init__(self,dic_size,dim,size):
        super().__init__()
        # self.dc = dic_size
        self.dim = dim
        self.embedding = nn.Embedding(dic_size,dim)
        self.pe = positional_encoding(dim,size)

    def forward(self,x):
        x = self.embedding(x) 
        #x = x* math.sqrt(self.dim)
        print(x.shape)
        #x = x + self.pe
        return x,pe



class AttentionHead(nn.Module):
    def __init__(self,l_layer):
        super().__init__()
        
        self.l1 = nn.Linear(l_layer,l_layer)
        self.l2 = nn.Linear(l_layer,l_layer)
        self.l3 = nn.Linear(l_layer,l_layer)
    def forward(self,x):
        q = f.relu(self.l1(x))
        k = f.relu(self.l2(x))
        v = f.relu(self.l3(x))
        return(x,q,k,v)


input = torch.LongTensor([[1,2,4],[4,3,2]])
data = input
dic_size = 1000;embed_size=512;attn_layer_size = embed_size
el = EmbeddingLayer(dic_size,embed_size,input.shape[1])
# print(type(data))
embed,pe = el(data)
print(embed)
attnhead = AttentionHead(attn_layer_size)
# print(attnhead(embed)[1])
# attnhead(embed)[1]

torch.Size([2, 3, 512])
tensor([[[-0.0382,  0.7443, -0.7847,  ..., -0.3645, -1.9231, -0.4937],
         [ 1.9632,  0.8879,  0.8878,  ..., -0.4422,  0.0637,  0.5649],
         [ 0.0391,  0.8868,  0.8470,  ...,  0.4675, -1.5627,  1.2016]],

        [[ 0.0391,  0.8868,  0.8470,  ...,  0.4675, -1.5627,  1.2016],
         [ 1.6990,  1.3319, -0.3974,  ..., -1.2178,  0.6342, -0.0397],
         [ 1.9632,  0.8879,  0.8878,  ..., -0.4422,  0.0637,  0.5649]]],
       grad_fn=<EmbeddingBackward0>)
